In [1]:
# keras imports for the dataset and building our neural network
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils


def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



### References: https://towardsdatascience.com/how-to-create-custom-real-time-plots-in-deep-learning-ecbdb3e7922f
### References: https://www.analyticsvidhya.com/blog/2020/11/tutorial-how-to-visualize-feature-maps-directly-from-cnn-layers/

class TrainingVis(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.acc = []
        self.f1score = []
        self.precision = []
        self.recall = []
        self.categorical_accuracy = []
        self.logs = []

    def on_train_batch_end(self,batch,logs={}):

        pass

        
        # if batch%100 ==0:
        #     layer_names = [layer.name for layer in model.layers]
        #     layer_outputs = [layer.output for layer in model.layers]
        #     feature_map_model = Model(inputs=model.input, outputs=layer_outputs)

        #     # create a feature map of dimension [1 : im.height : im.width : num_filters ]
        #     #feature_maps = feature_map_model.predict(X_sample)
        #     feature_maps = feature_map_model.predict(X_sample)

        #     clear_output(wait=True)

        #     for layer_name, feature_map in zip(layer_names, feature_maps):  

        #         if len(feature_map.shape) == 4:
        #             k = feature_map.shape[-1]  
        #             size=feature_map.shape[1]
        #             image_belt = np.zeros((size,size*k))


        #             for i in range(k):

        #                 # for every filter, visualize the activation    
        #                 feature_image = feature_map[0, :, :, i]
        #                 feature_image-= feature_image.mean()
        #                 feature_image/= feature_image.std ()
        #                 feature_image*=  25
        #                 feature_image+= 50

        #                 # Threshold the feature map to become a binary image 
        #                 feature_image = feature_image < 80
        #                 image_belt[:, i * size : (i + 1) * size] = feature_image 

        #             scale = 20. / k
        #             plt.figure( figsize=(scale * k, scale) )
        #             plt.title ( layer_name )
        #             plt.grid  ( False )
        #             plt.imshow( image_belt, aspect='auto')
        #             plt.show()


# to calculate accuracy
from sklearn.metrics import accuracy_score

dataset = 'fashion'

if dataset == 'digits':
    # loading the dataset

    (X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()


if dataset == 'fashion':
    X_train, y_train = load_mnist('/Users/andrewweng/developer/cnn-filter-metrics/data/fashion-mnist/data/fashion', kind='train')
    X_test, y_test = load_mnist('/Users/andrewweng/developer/cnn-filter-metrics/data/fashion-mnist/data/fashion', kind='t10k')
    print(X_train.shape)

# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

X_sample = X_train[22,:,:,:].reshape(1,28,28,1)



print(X_sample.shape)

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(50, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

metrics = [
    
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy")
]
plot_metrics = TrainingVis()

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics = metrics,optimizer='adam')







(60000, 784)
(1, 28, 28, 1)
Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


In [2]:
layer_names = [layer.name for layer in model.layers]
layer_outputs = [layer.output for layer in model.layers]
print(layer_outputs)

[<KerasTensor: shape=(None, 6, 6, 25) dtype=float32 (created by layer 'conv2d')>, <KerasTensor: shape=(None, 6, 6, 25) dtype=float32 (created by layer 'max_pooling2d')>, <KerasTensor: shape=(None, 900) dtype=float32 (created by layer 'flatten')>, <KerasTensor: shape=(None, 50) dtype=float32 (created by layer 'dense')>, <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_1')>]


In [ ]:
# training the model example by example, to wath the feature map change as weights are learned.

from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("simple_CNN", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(X_train, Y_train, batch_size=100, epochs=10, validation_data=(X_test, Y_test),verbose=1,callbacks=[early,checkpoint,plot_metrics])

model.predict()






In [ ]:
import matplotlib.pyplot as plt

print(hist.history)

plt.plot(hist.history["categorical_accuracy"])
plt.plot(hist.history['val_categorical_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()